In [2]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [4]:
import re
import PyPDF2

def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text

document_text = read_pdf('Umair_Ahmed.pdf')
preprocessed_text = preprocess_text(document_text)


In [5]:
def split_into_chunks(text, chunk_size=500):
    chunks = []
    words = text.split()
    for i in range(0, len(words), chunk_size):
        chunk = ' '.join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

document_chunks = split_into_chunks(preprocessed_text)


In [9]:
pip install pinecone-client


  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0


In [ ]:
import pinecone

pinecone.init(api_key='<YOUR_API_KEY>')
index = pinecone.Index("document_chunks_index")

# Assuming you have a function to convert text to embeddings called text_to_embedding()
# Replace this with the actual function or technique you're using for embedding
def text_to_embedding(text):
    # Your embedding logic here
    return embedding

for i, chunk in enumerate(document_chunks):
    embedding = text_to_embedding(chunk)
    index.upsert(str(i), embedding)


In [ ]:
def process_query(query, index, num_results=3):
    query_embedding = text_to_embedding(preprocess_text(query))
    results = index.query(queries=[query_embedding], top_k=num_results)
    relevant_chunk_indices = [int(result.id) for result in results]
    return relevant_chunk_indices

def chatbot(query):
    relevant_chunk_indices = process_query(query, index)
    for index in relevant_chunk_indices:
        if query in document_chunks[index]:
            return document_chunks[index]
    return "I don't know the answer."

# Example usage:
user_query = "What is the capital of France?"
response = chatbot(user_query)
print(response)
